In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

# Functions

In [ ]:
# import reusable functions from utils directory
import sys
sys.path.append('../../utils')
import my_utils

## Read and clean data

# Final Demo DF

In [ ]:
final_demo_df = pd.read_csv('../../data/raw/df_final_demo.txt')
final_demo_df.head()

In [ ]:
# clean column names
my_utils.clean_column_names(final_demo_df)

# inspect dataframe
my_utils.inspect_dataframe(final_demo_df)

In [ ]:
# change column names for standardization
new_column_names_final_demo_df = {
    'client_id': 'client_id',
    'clnt_tenure_yr': 'client_tenure_years',
    'clnt_tenure_mnth': 'client_tenure_months',
    'clnt_age': 'client_age',
    'gendr': 'gender',
    'num_accts': 'num_accounts',
    'bal': 'balance',
    'calls_6_mnth': 'calls_last_6_months',
    'logons_6_mnth': 'logons_last_6_months'
}

final_demo_df.rename(columns=new_column_names_final_demo_df, inplace=True)

In [ ]:
# check unique values
my_utils.check_unique_and_empty(final_demo_df)

In [ ]:
final_demo_df.head()

In [ ]:
# handle unique values in 'gender' column
print(final_demo_df['gender'].unique())

replacement_dict_gender = {
    'U': 'Unspecified',
    'M': 'Male',
    'F': 'Female',
    'X': 'Unspecified',
    np.nan: 'Unspecified'
}

# Apply the function to replace the gender values
final_demo_df['gender'] = final_demo_df['gender'].replace(replacement_dict_gender)

In [ ]:
# convert floats to ints in several columns
my_utils.floats_to_ints(final_demo_df, 'client_tenure_years')
my_utils.floats_to_ints(final_demo_df, 'client_tenure_months')
my_utils.floats_to_ints(final_demo_df, 'num_accounts')
my_utils.floats_to_ints(final_demo_df, 'calls_last_6_months')
my_utils.floats_to_ints(final_demo_df, 'logons_last_6_months')
my_utils.floats_to_ints(final_demo_df, 'client_age')
my_utils.check_unique_and_empty(final_demo_df)

In [ ]:
# drop empty values
final_demo_df = final_demo_df.dropna()
# check_unique_and_empty(final_demo_df)

# initial rows: 70609, rows after dropping empty values: 70594

final_demo_df.head()

In [ ]:
# generate clean csv file
# final_demo_df.to_csv('final_demo_df_clean.csv', index=False)
my_utils.inspect_dataframe(final_demo_df)

# Final Experiment Clients DF

In [ ]:
final_experiment_clients_df = pd.read_csv('../../data/raw/df_final_experiment_clients.txt')
final_experiment_clients_df.head()

In [ ]:
# clean column names
my_utils.clean_column_names(final_experiment_clients_df)

# inspect df
my_utils.inspect_dataframe(final_experiment_clients_df)
my_utils.check_unique_and_empty(final_experiment_clients_df)

In [ ]:
# check unique values
my_utils.check_unique_and_empty(final_experiment_clients_df)

In [ ]:
# replace empty values with 'unknown'

final_experiment_clients_df = final_experiment_clients_df.apply(lambda col: col.apply(lambda x: 'Unknown' if pd.isna(x) or x == '' else x))

my_utils.check_unique_and_empty(final_experiment_clients_df)

In [ ]:
# generate clean csv file
# final_experiment_clients_df.to_csv('final_experiment_clients_df.csv', index=False)

### Variation Count

In [ ]:
variation_percentages = final_experiment_clients_df['variation'].value_counts(normalize=True) * 100

print(variation_percentages)

In [ ]:
variation_counts = final_experiment_clients_df['variation'].value_counts()

variation_percentages = variation_counts / variation_counts.sum() * 100

colors = ['coral', 'lightgreen', 'lightblue']

plt.figure(figsize=(8, 6))
sns.barplot(x=variation_counts.index, y=variation_counts.values, palette=colors)

plt.title('Test vs Control Variation', fontsize=16)
plt.xlabel('Variation', fontsize=12)
plt.ylabel('Count', fontsize=12)

# annotate each bar with the percentage
for i, p in enumerate(plt.gca().patches):
    height = p.get_height()
    # add annotation with the percentage on top of each bar
    plt.text(p.get_x() + p.get_width() / 2, height + 0.1, f'{variation_percentages.iloc[i]:.1f}%', 
             ha='center', va='bottom', fontsize=12, color='black')

plt.show()

## Questions:

1. Primary clients using the online process
2. Are the primary clients younger or older?
3. Are the primary clients newer or long-standing?

In [ ]:
# clients using the online process (logons_last_6_months > 5)
primary_clients = final_demo_df[final_demo_df['logons_last_6_months'] > 5]

# We can compare the average age of primary clients vs. the full dataset
avg_age_all_clients = round(final_demo_df['client_age'].mean())
avg_age_primary_clients = round(primary_clients['client_age'].mean())


# We can compare the average tenure in years of primary clients vs. the full dataset
avg_tenure_all_clients = round(final_demo_df['client_tenure_years'].mean())
avg_tenure_primary_clients = round(primary_clients['client_tenure_years'].mean())

# Output the results
print(f'Primary clients:\n{primary_clients[['client_id', 'client_age', 'client_tenure_years', 'logons_last_6_months']]}\n')
print(f'Average age of all clients: {avg_age_all_clients}')
print(f'Average age of primary clients: {avg_age_primary_clients}')
print(f'Are primary clients younger or older? {'Younger' if avg_age_primary_clients < avg_age_all_clients else 'Older'}\n')
print(f'Average tenure of all clients: {avg_tenure_all_clients}')
print(f'Average tenure of primary clients: {avg_tenure_primary_clients}')
print(f'Are primary clients newer or long-standing? {'Newer' if avg_tenure_primary_clients < avg_tenure_all_clients else 'Long-standing'}')

## Combine DFs

- By now, `final_demo_df` has **70594 rows**, and `final_experiment_clients_df` has **70609 rows**.

In [ ]:
# check how many client ids match

client_ids_df1 = set(final_demo_df['client_id'])
client_ids_df2 = set(final_experiment_clients_df['client_id'])

# find the intersection
matching_ids = client_ids_df1.intersection(client_ids_df2)
num_matching_ids = len(matching_ids)

print(f'Matching client ids in both dataframes: {num_matching_ids}')

In [ ]:
# merge dataframes based on 'client_id', adding the 'variation' column
merged_variations_final_demo_df = pd.merge(final_demo_df, final_experiment_clients_df[['client_id', 'variation']], on='client_id', how='left')

# sort values by the original index
merged_variations_final_demo_df = merged_variations_final_demo_df.sort_index()

print(merged_variations_final_demo_df)

In [ ]:
# import to csv 
# merged_variations_final_demo_df.to_csv('../../data/clean/merged_final_demo_final_experiment_clients_df.csv', index=False)

In [ ]:
combined_cleaned_df = pd.read_csv('../../data/raw/combined_cleaned_data.csv')
combined_cleaned_df.head()

In [ ]:
my_utils.check_unique_and_empty(combined_cleaned_df)

In [ ]:
my_utils.inspect_dataframe(combined_cleaned_df)

In [ ]:
print(combined_cleaned_df['process_step'].unique())

In [ ]:
# Question - do we want to change this to Step 1, Step 2, and Finished?

In [ ]:
combined_cleaned_df['date_time'] = pd.to_datetime(combined_cleaned_df['date_time'], errors='coerce')

# check if any dates couldn't be converted (i.e., they are NaT)
inconsistent_dates = combined_cleaned_df['date_time'].isna().sum()

# If inconsistent_dates > 0, then there are invalid or mismatched date formats
if inconsistent_dates > 0:
    print(f'There are {inconsistent_dates} inconsistent or invalid date formats in the column.')
else:
    print('All dates in the column have the same format.')

print(combined_cleaned_df[['date_time']].head())

In [ ]:
my_utils.inspect_dataframe(combined_cleaned_df)

In [ ]:
## Find out KPI measurements

In [ ]:
# make a copy
kpi_df = combined_cleaned_df.copy()

In [ ]:
kpi_df.head()

In [ ]:
# drop columns
kpi_df = kpi_df.drop(columns=['visitor_id', 'visit_id'])

In [ ]:
# change the naming of the steps to numeric values to ensure order
replacement_dict_steps = {
    'start' : 0,
    'step_1' : 1,
    'step_2' : 2,
    'step_3' : 3,
    'confirm' : 4
}

kpi_df['process_step'] = kpi_df['process_step'].map(replacement_dict_steps)

In [ ]:
print(kpi_df)

# How many clients in total have finished?

In [ ]:
# filter rows where process_step is 'Finish'
clients_finished = kpi_df[kpi_df['process_step'] == 4]

# print(clients_finished)
total_unique_clients = kpi_df['client_id'].nunique()
# find unique client ids that finished
unique_clients_finished = clients_finished['client_id'].nunique()

print(f'Clients who finished the process: {unique_clients_finished} out ouf {total_unique_clients}.')

## How much time has been spent on each step (on average)?

In [ ]:
# sort df by client_id and date_time
kpi_df = kpi_df.sort_values(by=['client_id', 'date_time'])
# print(kpi_df)

kpi_df.head(50)

In [ ]:
# calculate time difference between steps per client
kpi_df['time_diff'] = kpi_df.groupby('client_id')['date_time'].diff()

# drop first row per client (contains no information)
kpi_df = kpi_df.dropna(subset=['time_diff'])
display(kpi_df)

In [ ]:
# calculate average time spent on each step
avg_time_per_step = kpi_df.groupby('process_step')['time_diff'].mean()

# Cconvert to total seconds
avg_time_in_seconds = avg_time_per_step.dt.total_seconds()

# round up to the nearest second to remove decimals
avg_time_in_seconds_rounded = np.ceil(avg_time_in_seconds).astype(int)

# convert back to timedelta
avg_time_per_step_rounded = pd.to_timedelta(avg_time_in_seconds_rounded, unit='s')

# re-map process_step index for readability
avg_time_per_step_rounded.index = avg_time_per_step_rounded.index.map({0: 'Start', 1: 'Step 1', 2: 'Step 2', 3: 'Step 3', 4: 'Finish'})

print('Average time spent on each step:\n')
print(avg_time_per_step_rounded)